# Project: Motion Detection using OpenCV

This project demonstrates a simple approach to motion detection using Python and OpenCV.  
It works by comparing video frames to identify areas of motion (like people moving), and highlighting them with rectangles.

## Steps Breakdown:

1. **Frame Differencing**  
   Compare the current video frame with the previous frame to detect changes (i.e., motion).

2. **Gaussian Blurring**  
   Apply Gaussian Blur to reduce noise caused by lighting changes or minor camera vibrations.  
   This smooths out the image and helps avoid detecting unnecessary small movements.

3. **Thresholding**  
   Convert the frame difference into a binary image — white for motion, black for no motion.

4. **Dilation**  
   Enlarge the white areas (motion zones) to make contours more visible and fill small gaps.

5. **Finding Contours**  
   Detect the outlines (contours) of motion areas from the thresholded image.

6. **Filtering Small Contours**  
   Remove tiny contours that are too small to be humans or meaningful objects.

7. **Drawing Rectangles**  
   For each remaining contour, draw a bounding rectangle on the frame to show the detected motion.




In [8]:
import cv2
import numpy as np

# Open video file
cap = cv2.VideoCapture(0)

# Get video frame size
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set up video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter("output.avi", fourcc, 5.0, (1280, 720))

# Read initial frames
ret, frame1 = cap.read()
ret, frame2 = cap.read()

print(f"Frame size: {frame1.shape}")

while cap.isOpened() and ret:
    # Step 1: Frame differencing
    diff = cv2.absdiff(frame1, frame2)

    # Step 2: Grayscale
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    # Step 3: Gaussian Blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Step 4: Thresholding
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)

    # Step 5: Dilation
    dilated = cv2.dilate(thresh, None, iterations=3)

    # Step 6: Find contours
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Step 7: Process contours
    for contour in contours:
        if cv2.contourArea(contour) < 800:
            continue

        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame1, "Status: Movement", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    # Resize and write frame
    image = cv2.resize(frame1, (1280, 720))
    out.write(image)

    # Show the frame
    cv2.imshow("feed", frame1)

    # Prepare for next iteration
    frame1 = frame2
    ret, frame2 = cap.read()

    # Exit on ESC
    if cv2.waitKey(40) == 27:
        break

# Cleanup
cap.release()
out.release()
cv2.destroyAllWindows()


Frame size: (480, 640, 3)
